# Scapy built-in arping method

In [1]:
import scapy.all as scapy

In [2]:
scapy.arping('10.0.2.1/24')

Begin emission:
Finished sending 256 packets.

Received 4 packets, got 4 answers, remaining 252 packets
  52:54:00:12:35:00 10.0.2.1
  08:00:27:fc:63:31 10.0.2.3
  52:54:00:12:35:00 10.0.2.2
  08:00:27:10:b8:d0 10.0.2.4


(<ARPing: TCP:0 UDP:0 ICMP:0 Other:4>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:252>)

# Crafting an ARP broadcast packet

In [3]:
scapy.ls(scapy.ARP())

hwtype     : XShortField                         = 1               (1)
ptype      : XShortEnumField                     = 2048            (2048)
hwlen      : ByteField                           = 6               (6)
plen       : ByteField                           = 4               (4)
op         : ShortEnumField                      = 1               (1)
hwsrc      : ARPSourceMACField                   = '08:00:27:74:17:d4' (None)
psrc       : SourceIPField                       = '10.0.2.15'     (None)
hwdst      : MACField                            = '00:00:00:00:00:00' ('00:00:00:00:00:00')
pdst       : IPField                             = '0.0.0.0'       ('0.0.0.0')


In [4]:
arp_request = scapy.ARP()
arp_request.pdst='10.0.2.1'
arp_request.summary()

'ARP who has 10.0.2.1 says 10.0.2.15'

In [5]:
scapy.ls(scapy.Ether())

dst        : DestMACField                        = 'ff:ff:ff:ff:ff:ff' (None)
src        : SourceMACField                      = '08:00:27:74:17:d4' (None)
type       : XShortEnumField                     = 36864           (36864)


In [6]:
ether_broadcast_frame = scapy.Ether()
ether_broadcast_frame.dst = 'ff:ff:ff:ff:ff:ff'
print(ether_broadcast_frame.summary())

08:00:27:74:17:d4 > ff:ff:ff:ff:ff:ff (0x9000)


In [7]:
arp_packet = ether_broadcast_frame/arp_request
arp_packet.summary()

'Ether / ARP who has 10.0.2.1 says 10.0.2.15'

In [8]:
arp_packet.show()

###[ Ethernet ]### 
  dst       = ff:ff:ff:ff:ff:ff
  src       = 08:00:27:74:17:d4
  type      = 0x806
###[ ARP ]### 
     hwtype    = 0x1
     ptype     = 0x800
     hwlen     = 6
     plen      = 4
     op        = who-has
     hwsrc     = 08:00:27:74:17:d4
     psrc      = 10.0.2.15
     hwdst     = 00:00:00:00:00:00
     pdst      = 10.0.2.1



In [9]:
answered, unanswered = scapy.srp(arp_packet, timeout=1)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [10]:
answered.summary()

Ether / ARP who has 10.0.2.1 says 10.0.2.15 ==> Ether / ARP is at 52:54:00:12:35:00 says 10.0.2.1 / Padding


# Crafting our ARP scanning method

In [11]:
def arp_scan(target):
    arp = scapy.ARP(pdst=target)
    ether = scapy.Ether(dst='ff:ff:ff:ff:ff:ff')
    ans, unans = scapy.srp(ether/arp, timeout=1, retry=10, verbose=False)
    
    print()
    for item in ans:
        print('==================================================\n')
        item[1].show()
    print('==================================================')
    
    results = []
    for item in ans:
        results.append({'IP': item[1].psrc, 'MAC': item[1].hwsrc})
    return results

In [12]:
results = arp_scan('10.0.2.1/24')



###[ Ethernet ]### 
  dst       = 08:00:27:74:17:d4
  src       = 52:54:00:12:35:00
  type      = 0x806
###[ ARP ]### 
     hwtype    = 0x1
     ptype     = 0x800
     hwlen     = 6
     plen      = 4
     op        = is-at
     hwsrc     = 52:54:00:12:35:00
     psrc      = 10.0.2.1
     hwdst     = 08:00:27:74:17:d4
     pdst      = 10.0.2.15
###[ Padding ]### 
        load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


###[ Ethernet ]### 
  dst       = 08:00:27:74:17:d4
  src       = 52:54:00:12:35:00
  type      = 0x806
###[ ARP ]### 
     hwtype    = 0x1
     ptype     = 0x800
     hwlen     = 6
     plen      = 4
     op        = is-at
     hwsrc     = 52:54:00:12:35:00
     psrc      = 10.0.2.2
     hwdst     = 08:00:27:74:17:d4
     pdst      = 10.0.2.15
###[ Padding ]### 
        load      = '\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


###[ Ethernet ]### 
  dst       = 08:00:27:74:17:d4
  src       = 08:00:

In [13]:
import pandas as pd
print(pd.DataFrame(results))

         IP                MAC
0  10.0.2.1  52:54:00:12:35:00
1  10.0.2.2  52:54:00:12:35:00
2  10.0.2.3  08:00:27:fc:63:31
3  10.0.2.4  08:00:27:10:b8:d0
